# シグナリングゲーム（ポリアの壺）

このノートブックでは，ポリアの壺モデルを用いた2体の強化学習エージェント（送信者と受信者）を同時に学習させて，シグナリングシステムが獲得される過程を観察する実験を行います．

まず，必要なパッケージをインポートします．この実験では，ほとんど外部のライブラリを使いませんので，インポートするのは `numpy` だけです．

In [ ]:
import numpy as np
rng = np.random.default_rng()

まず，シグナリングゲームを表現する強化学習環境（`SignalingGameEnv`）を定義します．
状態数（`num_states`），信号数（`num_messages`），状態の確率分布（`state_dist`）を指定して環境を作成できるようにしています．
環境の初期化を行う `__init__` メソッドの他に，以下の3つのメソッドを定義しています．

- `reset`: 最初からゲームプレイを行えるように環境をリセットします．この時，送信者に与えられる状態がランダムに決定されます．
- `observe`: エージェントに与えられる観測を返します．
- `step`: エージェントが選択した行動を受け取って，ゲームを進めます．

In [ ]:
class SignalingGameEnv:
    def __init__(self, num_states, num_messages, state_dist):
        self.num_states = num_states
        self.num_messages = num_messages
        self.state_dist = state_dist
        self.agents = ['sender', 'receiver']
        self.reset()

    def reset(self):
        self.state = rng.choice(self.num_states)
        self.done = False
        self.agent_selection = 'sender'

    def observe(self, agent):
        if agent == 'sender':
            return self.state
        elif agent == 'receiver':
            return self.message

    def step(self, action):
        assert self.done == False, 'WARNING: The game is done. Call reset().'
        self.rewards = {'sender': 0.0, 'receiver': 0.0}
        agent = self.agent_selection
        if agent == 'sender':
            self.message = action
            self.agent_selection = 'receiver'
        elif agent == 'receiver':
            self.action = action
            if self.state == action:
                self.rewards['sender'] = 1.0
                self.rewards['receiver'] = 1.0
            self.done = True

次に，ポリアの壺モデルに基づく強化学習エージェントのクラス（`UrnAgent`）を定義します．
観測の種類数（`num_obs`，送信者の場合は状態数，受信者の場合は信号数に対応）と，行動の種類数（`num_actions`，送信者の場合は信号数，受信者の場合は状態数に対応）を指定して作成できるようにしています．初期化を行う `__init__` メソッドの他に，以下の4つのメソッドを定義しています．

- `get_action`: 観測を受け取って，行動を選択して返します．
- `store_buffer`: 観測と状態のペアを受け取って，経験として訓練バッファに保存します．
- `update_reward`: 次の自分のターンが回ってくる前に，他のエージェントの行動によって自分が報酬を受け取れる場合があります（例えばシグナリングゲームの場合，送信者が行動した時点では報酬はありませんが，次に受信者が適切な行動をした場合には，送信者にも報酬が与えられます）．これを，直近の行動に対する報酬とみなして，訓練バッファの内容を更新します．
- `train`: 現在の訓練バッファの内容を使って，方策を更新します．

In [ ]:
class UrnAgent:
    def __init__(self, num_obs, num_actions):
        self.num_obs = num_obs
        self.num_actions = num_actions
        self.urn_balls = [np.ones(num_actions, dtype=float) for _ in range(num_obs)]
        self.urn_sum_balls = [num_actions for _ in range(num_obs)]
        self.train_buf = []

    def get_action(self, obs):
        p = self.urn_balls[obs] / self.urn_sum_balls[obs]
        return rng.choice(np.arange(self.num_actions), p=p)

    def store_buffer(self, obs, action):
        self.train_buf.append([obs, action, 0]) # 観測，行動，報酬の組

    def update_reward(self, reward):
        if len(self.train_buf) > 0:
          self.train_buf[-1][2] += reward

    def train(self):
        for obs, action, reward in self.train_buf:
            self.urn_balls[obs][action] += reward
            self.urn_sum_balls[obs] += reward
        self.train_buf = []

以下のセルで，具体的に環境と強化学習モデルを定義します．
環境のパラメータを変えて実験してみましょう．

In [ ]:
num_states = 2          # 状態数
num_messages = 2        # 信号数
# 状態の確率分布
state_dist = [1.0 / num_states] * num_states
# state_dist = [0.5, 0.5]
env = SignalingGameEnv(num_states, num_messages, state_dist)

sender = UrnAgent(env.num_states, env.num_messages)
receiver = UrnAgent(env.num_messages, env.num_states)
agents = {'sender': sender, 'receiver': receiver}

total_timesteps = 20000  # ゲームプレイ回数
n_steps = 5             # 訓練データバッファのサイズ

以下のコードで，2体の強化学習モデルの学習を行います．

In [ ]:
log_rewards = {'sender': [], 'receiver': []}  # あとで学習過程を確認するためのログ
for timestep in range(total_timesteps):
    # ゲームプレイ開始
    env.reset()
    while not env.done:
        agent_name = env.agent_selection
        obs = env.observe(agent_name)
        action = agents[agent_name].get_action(obs)
        env.step(action)
        agents[agent_name].store_buffer(obs, action)
        for _agent_name in env.agents:
            reward = env.rewards[_agent_name]
            agents[_agent_name].update_reward(reward)
    # ゲームプレイ終了．方策を更新する
    if (timestep + 1) % n_steps == 0:
        for _agent_name in env.agents:
            mean_reward = np.mean([r for o, a, r in agents[_agent_name].train_buf])
            log_rewards[_agent_name].append(mean_reward) # ロギング
            agents[_agent_name].train()

以下のコードで，壺の中に入っているボールの数を確認する．

In [ ]:
# Print agent's urn
for agent_name in env.agents:
    agent = agents[agent_name]
    obs_name, action_name = ('state', 'message') if agent_name == 'sender' else ('message', 'action')
    print(f'{agent_name} urn:')
    for obs, balls in enumerate(agent.urn_balls):
        print('{0:>7} {1:>2}:'.format(obs_name, obs))
        for action, n in enumerate(balls):
            print('  {0:>7} {1:>2}: {2:>6}'.format(action_name, action, n))
        print()

以下のコードで，学習した結果の方策を用いたときの，ゲームプレイごとの平均報酬を評価します．
コミュニケーションが成功すれば報酬1，失敗すれば報酬0なので，平均報酬の値は，コミュニケーションの成功率ということになります．

In [ ]:
# Evaluate policy
n_eval_episodes = 100
sender_rewards = []
receiver_rewards = []
for _ in range(n_eval_episodes):
    env.reset()
    while not env.done:
        agent_name = env.agent_selection
        obs = env.observe(agent_name)
        action = agents[agent_name].get_action(obs)
        env.step(action)
    sender_rewards.append(env.rewards['sender'])
    receiver_rewards.append(env.rewards['receiver'])
print(f'Sender average reward: {np.mean(sender_rewards):.2f}')
print(f'Receiver average reward: {np.mean(receiver_rewards):.2f}')

以下のコードでは，学習過程における報酬のログをTensorboardで確認できる形式で保存している．

In [ ]:
from torch.utils.tensorboard import SummaryWriter
settings = f'states{env.num_states}_messages{env.num_messages}_state_dist{env.state_dist}'
writer = SummaryWriter(log_dir=f'./tb/{settings}')
for i, (r_s, r_r) in enumerate(zip(log_rewards['sender'], log_rewards['receiver'])):
    writer.add_scalar("sender_reward", r_s, i * n_steps)
    writer.add_scalar("receiver_reward", r_r, i * n_steps)
writer.close()

以下のコマンドでTensorboardが起動する．
このセルは一度だけ実行し，再実行しないことをお勧めする．
新しく追加したデータを再読み込みしたい場合，Tensorboardのコンソール上のリロードボタン（右上の方にあるボタン）を押してデータの再読み込みを行うようにするとよい．

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./tb

# 演習課題

- 状態数（`num_states`）と信号数（`num_messages`）を大きくして実験を行い，コミュニケーションの成功率がどのように変化するか調べてみよう．また，その時の壺の中のボールの分布を観察して，なぜコミュニケーションの成功率が変化したのか考察してみよう．
- 状態数が2，信号数が2のシグナリングゲームで，状態の確率が偏った設定（`[0.7, 0.3]` や `[0.8, 0.2]`，`[0.9, 0.1]` など）で実験を行い，コミュニケーションの成功率がどのように変化するか調べてみよう．また，その時の壺の中のボールの分布を観察して，なぜコミュニケーションの成功率が変化したのか考察してみよう．
- 信号数を，状態数とは異なる数に設定して実験を行い，コミュニケーションの成功率がどのように変化するか調べてみよう．信号数が状態数よりも小さい場合と，大きい場合で，それぞれ壺の中のボールの分布がどのように変化するか観察して，結果の理由を考察してみよう．